In [1]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [8]:
import os

os.chdir("..")

from utils.utils import set_seed
set_seed()

### Data understand

In [9]:
import pandas as pd

In [ ]:
ds_train = pd.read_csv('../data/avila/avila-tr.txt', header=None)
ds_test = pd.read_csv('../data/avila/avila-ts.txt', header=None)